In [1]:
import findspark
findspark.init('/home/daniel/spark-3.2.1-bin-hadoop3.2')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lr_example").getOrCreate()

22/04/09 06:39:23 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.234.128 instead (on interface ens33)
22/04/09 06:39:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/09 06:39:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/09 06:39:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv("Ecommerce_Customers.csv", header=True, inferSchema=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


# Assemble features

In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

### Only gonna use numerical data as features for now

In [10]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [11]:
assembler = VectorAssembler(
    inputCols=[
        'Avg Session Length',
        'Time on App',
        'Time on Website',
        'Length of Membership'
    ],
    outputCol="features" # name of the output column
)

In [13]:
### Transform original data into assembler

In [12]:
output = assembler.transform(data)

In [14]:
output.head(1)
# features col is a DenseVector with all the numerical values

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [15]:
final_data = output.select("features", "Yearly Amount Spent")

In [16]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

### Train test split

In [17]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [18]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                342|
|   mean| 500.96758098413414|
| stddev|  79.60516765572532|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [19]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                158|
|   mean|  495.7348508400094|
| stddev|  78.81511766300855|
|    min| 298.76200786180766|
|    max|  744.2218671047146|
+-------+-------------------+



### Training

In [22]:
lr = LinearRegression(
    labelCol="Yearly Amount Spent"
)

In [23]:
model = lr.fit(train_data)

22/04/09 06:51:51 WARN Instrumentation: [d1acc040] regParam is zero, which might cause numerical instability and overfitting.
22/04/09 06:51:52 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/04/09 06:51:52 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/04/09 06:51:52 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [24]:
test_results = model.evaluate(test_data)

In [25]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  9.623025433698558|
|-0.5521779356370757|
| 10.439325939764899|
|  6.095229256852463|
| -8.342647522606967|
|-0.5685210436841999|
|-3.7979810962597185|
| 3.6438620778316135|
| -6.028009731690759|
| 3.7007056543503154|
| -15.11435496039644|
|-2.8008494492424916|
|-1.0620112338602894|
|-26.992580167994333|
| -8.319442730935975|
|-1.2653070202222239|
|-5.3121169154626955|
|-12.211318867057344|
| -2.698217504239892|
| -2.279114502886955|
+-------------------+
only showing top 20 rows



/home/daniel/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [26]:
test_results.rootMeanSquaredError

9.805122512391664

In [27]:
test_results.r2

0.9844244128589432

### Original data

In [28]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



### RMSE 9.8 vs AVG 499 (being off by 9.8 dollars is acutally good)
### R2 the model explains 98% of the variance of the data

# Deploy on new data

In [29]:
unlabeled_data = test_data.select("features")

In [30]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.5743636841713...|
|[30.7377203726281...|
|[30.9716756438877...|
|[31.1280900496166...|
|[31.2606468698795...|
|[31.2681042107507...|
|[31.3584771924370...|
|[31.5257524169682...|
|[31.5316044825729...|
|[31.5741380228732...|
|[31.5761319713222...|
|[31.6610498227460...|
|[31.6739155032749...|
|[31.7207699002873...|
|[31.7216523605090...|
|[31.7656188210424...|
|[31.8093003166791...|
|[31.8530748017465...|
|[31.8627411090001...|
+--------------------+
only showing top 20 rows



In [31]:
predictions = model.transform(unlabeled_data)

In [32]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 399.0173256389289|
|[30.5743636841713...|442.61659169370273|
|[30.7377203726281...|  451.341416256465|
|[30.9716756438877...|488.54338050004026|
|[31.1280900496166...| 565.5953342696616|
|[31.2606468698795...| 421.8951523006356|
|[31.2681042107507...|427.26851427008364|
|[31.3584771924370...| 491.5320883716438|
|[31.5257524169682...|449.99363654157264|
|[31.5316044825729...|432.81490007501225|
|[31.5741380228732...| 559.5236271209833|
|[31.5761319713222...| 544.0274334385708|
|[31.6610498227460...| 417.4203648137611|
|[31.6739155032749...|502.71764807787554|
|[31.7207699002873...| 547.0943762089589|
|[31.7216523605090...|349.04223365209486|
|[31.7656188210424...|501.86619855106983|
|[31.8093003166791...| 548.9832182298985|
|[31.8530748017465...|461.98334096659187|
|[31.8627411090001...| 558.5772556769336|
+--------------------+------------

for first customer prediction is 399 dollars etc.